### CS 210: Data Management for Data Science
#### Fall 2023

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [194]:
NAME = "Shivam Patel"
COLLABORATORS = "Me Me and Me"

---

## Homework 3: <font color="blue">Movie Recommendations</font>

### Overview
In this assignment, you'll implement a prototype movie
recommendation program by filling in the function implementations in
the given `hw3.py` template file.

**Make sure you implement your functions in `hw3.py`**. Do not implement
the functions in this notebook. They will be imported here from `hw3.py`, 
so you can test them directly. 

You can run your tests on the given input files, but you're
encouraged to make your own test files as well, to make sure that
you cover the various paths of logic in your functions. You don't
have to submit any of your test files.

You may assume that all parameter values given to your functions
will be valid. Also, in any function that requires the returned
values to be sorted or ranked, ties may be broken arbitrarily
between equal values.

You may assume that input files will be correctly formatted, and
data types will be as expected.

For all rating computations, do not round up (or otherwise modify)
the rating unless otherwise specified.

#### Input Files

**Ratings file**: A text file that contains movie ratings. Each line has the id of the user who rated the movie, the id of the movie, and its rating (range 0-5 inclusive). A movie can have multiple ratings from different users. A user can rate multiple movies but can rate a particular movie only once. 

**Movies file**: A text file that contains movies and their genres. Each line has a movie id, the name of the movie, the year it was released, and the genre of the movie (e.g, Action, Drama, etc.). 

There are sample movies and ratings files provided on Codebench. 

### Part 1: Reading Data

In [195]:
import hw3

In [196]:
from hw3 import read_movies_data, read_ratings_data

#### 1.1 (8 points) Read Movies File

Write a function `read_movies_data(f)` that takes in a movies file and returns a pandas DataFrame, where the index is the movie id and the columns are "title", "year", and "genre" (in that order). 


In [197]:
import pandas as pd
def read_movies_data(f):
    movies = pd.read_csv(f, sep='|', header=None, names=['title', 'year', 'genre'], index_col= 0)
    #movies = movies.set_index('ID')
    movies = movies.sort_index(ascending = True)

    #movies = movies.drop(columns=['ID'])
    #movies.columns = ['title', 'year', 'genre']
    #movies.drop('ID', axis=1)
    #movies.reset_index(drop=True, inplace=True)
    #movies.index += 1
    return movies
    #print(movies)
    pass


In [198]:
movies_df = read_movies_data("moviesSample.txt")

In [199]:
assert movies_df.shape == (9, 3)
assert movies_df.iloc[2, 2] == "Comedy"
assert movies_df.loc[6, "title"] == "Heat"
assert all(movies_df.columns == ["title", "year", "genre"])
assert list(movies_df.index) == [1, 2, 3, 4, 5, 6, 8, 9, 10]

#### 1.2 (8 points) Read Ratings File
Write a function `read_ratings_data(f)` that takes in a ratings
file name and returns a dictionary. The dictionary should have
the movie id as key, and the corresponding list of ratings as value.

In [200]:
def read_ratings_data(f):
    ratings_dict = {}
    
    with open(f, 'r') as file:
        for line in file:
            user, movie, rating = line.strip().split(',')
            movie = int(movie)
            rating = float(rating)
            
            if movie in ratings_dict:
                ratings_dict[movie].append(rating)
            else:
                ratings_dict[movie] = [rating]

        

    return ratings_dict
    #print(ratings_dict)

    pass

In [201]:
movie_ratings = read_ratings_data("ratingsSample.csv")


In [202]:
assert sum(movie_ratings[5]) == 24


### Part 2: Processing Data

In [203]:
from hw3 import get_movie, create_genre_dict, calculate_average_rating

#### 2.1 (8 points) Genre Dictionary

Write a function `create_genre_dict(df)` which takes a movies DataFrame as a parameter and returns a dictionary with each genre as a key and a list of movie ids as values. 

Example: 
```
{
    "Adventure": [1, 2, 8],
    "Comedy": [3, 4, 5],
    "Action": [6, 9, 10]
}
```

In [204]:
def create_genre_dict(movies_df):
    genre_dict = {}
    for i, row in movies_df.iterrows():
        genre = row['genre']
        movie_id = row.name
        if genre in genre_dict:
            genre_dict[genre].append(movie_id)
        else:
            genre_dict[genre] = [movie_id]
    return genre_dict
    
    #print(genre_dict)

    pass

In [205]:
genre_dict = create_genre_dict(movies_df)

#### 2.2 (8 points) Average Rating

Write a function `calculate_average_rating` which takes as input a ratings dictionary like that created in Part 1.2 and a movies DataFrame of the kind created in Part 1.1. The function should return a pandas `Series` where the index corresponds to the movie id and the value is the average of all the ratings for the movie. 

In [206]:
def calculate_average_rating(ratings_dict, movies_df): 
    average_ratings = pd.Series(index=movies_df.index, dtype=float)
    for id, ratings in ratings_dict.items():
        if id in average_ratings.index:
            average = sum(ratings) / len(ratings)
            average_ratings.at[id] = average
    #print(average_ratings)
    return average_ratings


    pass

In [207]:
avg_ratings = calculate_average_rating(movie_ratings, movies_df)

In [208]:
assert avg_ratings.loc[3] == 4.0

### Part 3: Recommendations

In [209]:
from hw3 import get_popular_movies, filter_movies, get_popular_in_genre
from hw3 import get_genre_rating, get_movie_of_the_year

#### 3.1 (8 points) Popularity Based

In services such as Netflix and Spotify, you often see
recommendations with the heading "Popular movies" or "Trending top
10".

Write a function `get_popular_movies` that takes as parameters a pandas Series
of movie-to-average rating (as created in Part 2.2) and an integer $n$ 
(default should be 10). 
The function should return a
pandas Series (movie_id:average rating, same structure as input) of top $n$ movies based on the average ratings. If there
are fewer than $n$ movies, it should return all movies in order of top
average ratings. The Series should be sorted in descending order by average rating. 
```
[In]: get_popular_movies(avg_ratings, 4)
[Out]: 
6    4.250000
5    4.000000
3    4.000000
1    3.833333
dtype: float64
```

In [210]:
def get_popular_movies(avg_ratings, n=10):
    sorted_ratings = avg_ratings.sort_values(ascending=False)
    return sorted_ratings.head(n)
    pass

In [211]:
get_popular_movies(avg_ratings, 4)

6    4.250000
3    4.000000
5    4.000000
1    3.833333
dtype: float64

In [212]:
pop_movies = get_popular_movies(avg_ratings, 4)
assert pop_movies.is_monotonic_decreasing
assert set(pop_movies.index) == {6, 5, 3, 1}

#### 3.2 (8 point) Threshold Rating

Write a function `filter_movies` that takes as parameters a pandas Series of
movie-to-average rating (same as for the popularity based
function above), and a threshold rating with default value of 3. The
function should filter movies based on the threshold rating, and
return a dictionary with same structure as the input. For example,
if the threshold rating is 3.5, the returned dictionary should have
only those movies from the input whose average rating is equal to or
greater than 3.5.

Example: 
```
[In]: filter_movies(avg_ratings, 3.2)

[Out]: 
1    3.833333
2    3.416667
3    4.000000
5    4.000000
6    4.250000
9    3.333333
dtype: float64
```

In [213]:
def filter_movies(avg_ratings, thres_rating=3):
    return avg_ratings[avg_ratings >= thres_rating]
    pass

In [214]:
filter_movies(avg_ratings, 3.2)

1    3.833333
2    3.416667
3    4.000000
5    4.000000
6    4.250000
9    3.333333
dtype: float64

In [215]:
good_movies = filter_movies(avg_ratings, 3.2)
assert good_movies.shape[0] == 6
assert all(good_movies.values >= 3.2)

#### 3.3 (8 points) Popularity & genre based

In most recommendation systems, genre of the movie/song/book plays
an important role. Often features like popularity, genre, artist are
combined to present recommendations to a user.

Write a function `get_popular_in_genre` which takes as parameters a genre, a genre-to-movies dictionary (as created in 2.1), a pandas Series of movie-to-average rating (as created in 2.2) and an integer $n$ (default 5), and returns the top $n$ movies for the given genre. The return should be a Series of movie-to-average rating that make the cut, i.e., the same format as the input. If there are fewer than $n$ movies, all movies for the genre are returned, in decreasing order by average rating. 
```
[In]: get_popular_in_genre("Adventure", genre_dict, avg_ratings, 2)
[Out]: 
1    3.833333
2    3.416667
dtype: float64
```

In [216]:
def get_popular_in_genre(genre, genre_to_movies, avg_ratings, n=5):
    if genre in genre_to_movies:
        movies_in_genre = genre_to_movies[genre]
        genre_ratings = avg_ratings.loc[movies_in_genre].sort_values(ascending=False)
        return genre_ratings.head(n)
    pass


In [217]:
get_popular_in_genre("Adventure", genre_dict, avg_ratings, 2)

1    3.833333
2    3.416667
dtype: float64

In [218]:
pop_in_genre = get_popular_in_genre("Adventure", genre_dict, avg_ratings, 2)
assert list(pop_in_genre.index) == [1, 2]
assert pop_in_genre.is_monotonic_decreasing

#### 3.4 (8 points) Genre Ratings

One important analysis for the content platforms is to determine
ratings by genre.

Write a function `get_genre_rating` that takes the same parameters as
`get_popular_in_genre` above, except for $n$, and returns the average
rating of the movies in the given genre.
```
[In]: get_genre_rating("Action", genre_dict, avg_ratings)
[Out]: 3.527777777777778
```

In [219]:
def get_genre_rating(genre, genre_to_movies, avg_ratings):
    if genre in genre_to_movies:
        movies = genre_to_movies[genre]
        genre_avg_rating = avg_ratings.loc[movies].mean()
        return genre_avg_rating
    pass

In [220]:
get_genre_rating("Action", genre_dict, avg_ratings)

3.527777777777778

In [221]:
genre_rating = get_genre_rating("Action", genre_dict, avg_ratings)
assert genre_rating > 3.525
assert genre_rating < 3.53

#### 3.5 (8 points) Movie of the Year

Write a function `get_movie_of_the_year` which takes as parameters a year, a movie-to-average rating Series (as created in 2.2), and a movies DataFrame (as created in 1.1)
and returns the title of the movie with the highest average rating 
for the given year. 
```
[In]: get_movie_of_the_year(1995, avg_ratings, movies_df)
[Out]: 'Heat'
```

In [222]:
def get_movie_of_the_year(year, avg_ratings, movies_df):
    #best_movie = movies_df[movies_df['year']==year]
    #highest_rating = avg_ratings[(movies_df[movies_df['year']==year]).index].idxmax()
    #highest_rating = movies_df.loc[(avg_ratings[(movies_df[movies_df['year']==year]).index].idxmax()), 'title']
    return movies_df.loc[(avg_ratings[(movies_df[movies_df['year']==year]).index].idxmax()), 'title']
    pass

In [223]:
assert get_movie_of_the_year(1995, avg_ratings, movies_df) == 'Heat'

### Part 4: User Focused

In [224]:
from hw3 import read_user_ratings, get_user_genre, recommend_movies

#### 4.1 **(8 points) User Ratings**

Read the ratings file to return a user-to-movies dictionary that maps user ID to the associated movies
and the corresponding ratings. Write a function named `read_user_ratings` for this, with the ratings
file as the parameter.

For example:
```
{
    u1: [(m1, r1), (m2, r2)],
    u2: [(m3, r3), (m8, r8)]
}
```
where `u1` is a user ID, `m1` is a movie ID, and `r1` is the corresponding rating. 



In [225]:
def read_user_ratings(f):
    user_dict = {}
    
    with open(f, 'r') as file:
        for line in file:
            user, movie, rating = line.strip().split(',')
            user = int(user)
            movie = int(movie)
            rating = float(rating)

            if user in user_dict:
                user_dict[user].append((movie, rating))
            else:
                user_dict[user] = [(movie, rating)]
    
    return user_dict

    pass

In [226]:
user_movies = read_user_ratings("ratingsSample.csv")

In [227]:
assert user_movies[42] == [(3, 4.0)]

#### 4.2 (10 points) User Genre

Write a function `get_user_genre` that takes as parameters a user id, the user-to-movies dictionary
(as created in Part 4.1 above), and the movies DataFrame (as created in Part 1.1), and returns
the top genre that the user likes based on the user’s ratings. Here, the top genre for the user will be
determined by taking the average rating of the movies genre-wise that the user has rated. If multiple
genres have the same highest ratings for the user, return any one of genres (arbitrarily) as the top
genre.

```
[In]: get_user_genre(6, user_movies, movies_df)
[Out]: 'Comedy'
```

In [228]:
def get_user_genre(user_id, user_to_movies, movies_df):
    user_dict = user_to_movies.get(user_id, [])
    genre_ratings = {}
    avg = {}
    
    for movie, rating in user_dict:
        genre = movies_df.loc[movie, 'genre']
        if genre in genre_ratings:
            genre_ratings[genre].append(rating)
        else:
            genre_ratings[genre] = [rating]
            
    for genre, ratings in genre_ratings.items():
        if len(ratings) > 0:
            avg_rating = sum(ratings) / len(ratings)
    
        avg[genre] = avg_rating
    return max(avg, key=avg.get)
    pass

In [229]:
get_user_genre(6, user_movies, movies_df)

'Comedy'

In [230]:
user_genre = get_user_genre(42, user_movies, movies_df)
assert user_genre == "Comedy"

#### 4.3 (10 points) User Recommendations

Recommend 3 most popular (highest average rating) movies from the user’s top genre that the user
has not yet rated. Write a function called `recommend_movies` for this, that takes as parameters a user id,
the user-to-movies dictionary (as created in Part 4.1 above), the movies DataFrame (as created
in Part 1.1), and the movie-to-average rating Series (as created in Part 2.2). The function should
return a pandas Series of movie-to-average rating, sorted in decreasing order by rating. If fewer than 3 movies make the cut, then return all
the movies that make the cut in order of top average ratings.
```
[In]: recommend_movies(6, user_movies, movies_df, avg_ratings)
[Out]: 
3    4.0
5    4.0
4    2.5
Name: avg_ratings, dtype: float64
```

In [231]:
def recommend_movies(user_id, user_to_movies, movies_df, avg_ratings):
    user_genre = get_user_genre(user_id, user_to_movies, movies_df)
    print(f"User {user_id}'s preferred genre: {user_genre}")

    unrated_movies = set(movies_df.index) - set([movie for movie, _ in user_to_movies.get(user_id, [])])

    genre_dict = create_genre_dict(movies_df)

    unrated_movies_in_genre = [movie for movie in unrated_movies if genre_dict.get(user_genre, []) and movie in genre_dict[user_genre]]

    recommended_movies = avg_ratings.loc[unrated_movies_in_genre].sort_values(ascending=False)

    print(f"Recommended movies: {recommended_movies}")
    return recommended_movies
    pass

In [232]:
recommend_movies(6, user_movies, movies_df, avg_ratings)

User 6's preferred genre: Comedy
Unrated movies: {1, 9}
Unrated movies in Comedy: []
Recommended movies: Series([], dtype: float64)


Series([], dtype: float64)